In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from kadboxes import *
from kadboxes.util import enable_viewer
enable_viewer("Pico Outside ARGB")

Overwriting auto display for cadquery Workplane and Shape


In [3]:
outer_sizes = (80, 80)
wall_thickness = 4

box = container(*outer_sizes, 40, wall_thickness=wall_thickness, with_snap=False)

pico_holder = mounting_box(51.5, 21.5, 8).union(pin_stands(47, 11.4, 3, .8, base_height=5))
pico_holder = pico_holder.translate((0,20,0))

power_holder = mounting_box(45.5, 23.5, 8)
power_holder = power_holder.translate((0,-20,0))

box = box.union(pico_holder).union(power_holder)


box = box.faces("<Y").workplane(centerOption="CenterOfMass").center(0, 10).rect(outer_sizes[0]*0.75, 5).extrude(2.5)
box = box.faces(">Y").workplane(centerOption="CenterOfMass").center(0, 10).rect(outer_sizes[0]*0.75, 5).extrude(2.5)
box = box.faces("<Y").workplane().center((outer_sizes[0]/2) + (wall_thickness), -5).hole(3)

# Bottom Holes - M12.07
hole_offset = outer_sizes[1] * .25
box = box.faces("<X").workplane(centerOption="CenterOfMass").center(hole_offset,-5).circle(12.5/2).cutBlind(-5)
box = box.faces("<X").workplane(centerOption="CenterOfMass").center(hole_offset+1,-5).circle(15.5/2).cutBlind(-3)

box = box.faces("<X").workplane(centerOption="CenterOfMass").center(-hole_offset,-5).circle(12.5/2).cutBlind(-5)
box = box.faces("<X").workplane(centerOption="CenterOfMass").center(-hole_offset-1,-5).circle(15.5/2).cutBlind(-3)

# Light Resistor
box = box.faces(">X").workplane(centerOption="CenterOfMass").center(outer_sizes[0]*0.4,-10).circle(6).extrude(3).faces(">X").rect(5.2, 2).cutBlind(-10)

wedge = cq.Workplane("XZ").moveTo(0, 0).lineTo(0, 10).lineTo(20, 3).lineTo(20,0).close().extrude(30)
wedge = wedge.faces("<Z").workplane(centerOption="CenterOfMass", offset=-5).rect(25,15).cutBlind(-30)
wedge = wedge.faces("<Z").workplane(centerOption="CenterOfMass").center(-2,0).hole(4)
box = box.union(wedge.translate(((outer_sizes[0]/2)+wall_thickness,15,-4)))

box

CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…

In [216]:
cq.exporters.export(box, 'outside-box.stl')


# LID BELOW

In [4]:
lid_size = tuple([x + wall_thickness*2 + 1 for x in outer_sizes])
lid_height = 20

lid = container(*lid_size, lid_height, wall_thickness=wall_thickness, with_snap=False)

lid = (
    lid
    .faces("<X")
    .workplane(centerOption="CenterOfMass")
    .center(0, 2)
    .rect(lid_size[0], lid_height)
    .cutBlind(-5)
)

lid = lid.faces("<Y").workplane(centerOption="CenterOfMass", offset=-1).center(-4, 4).rect(lid_size[0], 7).cutBlind("next")
lid = lid.faces(">Y").workplane(centerOption="CenterOfMass", offset=-1).center(4, 4).rect(lid_size[0], 7).cutBlind("next")

cq.exporters.export(lid, 'outside-box-lid.stl')

lid

CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…

In [5]:
ass = (
    cq.Assembly(name="box+lid")
    .add(box)
    .add(lid.rotate(
        (0, 0, 0), (1, 0, 0), 180
    ), loc=cq.Location(cq.Vector(0, 0, 40)))
)
ass

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.02s


CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…

In [133]:
bb = cq.Workplane("YX").workplane(offset=10).center(0,40).rect(100,30).extrude(-40)
bb = bb.intersect(lid)
cq.exporters.export(bb, 'outside-box-bb.stl')


# Now for the light sensor "nipple"

In [6]:
nipple = cq.Workplane("YZ").circle(6.5).extrude(8)
nipple = (
    nipple
    .faces("<X")
    .circle(8.5)
    .extrude(10)
    .cut(nipple)
)

ass = (
    cq.Assembly(name="box+nipple")
    .add(box)
    .add(nipple,loc=cq.Location(cq.Vector(44, 32, 7.5)), color=cq.Color("red"))
)
cq.exporters.export(nipple, 'outside-box-nipple.stl')
ass

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.01s


CadViewerWidget(anchor=None, cad_width=800, glass=False, height=600, pinning=False, theme='light', title=None,…